# Word2Vec + Models

In this document, we are trying to train a lot of different models using word2vec as an encriptor for the X (the descriptions).

The document is structred as follows:

- **Load data**
    - Descriptions (x)
    - Features (y)
    
- **Word2Vec (for descriptions)**
- **PCA (for features)**

- **Model(s)** training *make sure to comment the best results of each method with its hyperparameters*
    1. MLP
    2. XX

- **Accuracy in the validation set**


___

Ideas of things to try:

- N grams
- Keras
- Using Tags! [done]
- use tags and descriptions at the same time
- Ensambled method


- Only pairwise similarities

In [1]:
import os
import numpy as np
import gensim
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import string
import pandas as pd
import sklearn

from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA

In [2]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs


def parse_tags(tags):
    result = []
    for doc in tags:
        doc = doc.strip('\n').split('\n')
        cat_it = []
        if doc[0] == '':
            result.append('no tag')
        else:
            for tag in doc:
                split_tag = tag.split(':')
                cat_it.append(split_tag[0])
                cat_it.append(split_tag[1])
            parsed = (' ').join(list(set(cat_it)))
            result.append(parsed)
    return np.array(result)


# function to preprocess data
def preprocessing(data):
    stop_words = set(stopwords.words('english')) # find stop words in English language
    lemmatizer = WordNetLemmatizer() # declare nltk lemmatizer

    # iterate through every sentence and replace it by itself lemmatized, without punctuation and without stop words
    for i in range(len(data)):
        sentence_no_punct = ''
        # remove punctuation
        
        for char in data[i]:
            if char not in string.punctuation:
                sentence_no_punct = sentence_no_punct + char
        data[i] = sentence_no_punct

        word_tokens = word_tokenize(data[i])
    
        # remove stop words and lemmatize
        word_tokens = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words and len(word) > 1]
        word_tokens = [lemmatizer.lemmatize(word, 'v') for word in word_tokens]
        word_tokens = [lemmatizer.lemmatize(word, 'a') for word in word_tokens]
        
        # remove conjunction words
        word_tokens = [word for word in word_tokens if word[-2:] != 'nt']
        (data[i]) = ' '.join(word_tokens)
        
    return data

def StandardScaler (data):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(data)
    return scaler.transform(data)

## Load data

#### Descriptions

In [3]:
#DOCUMENTS

train_descs = parse_descriptions('cs5785-fall19-final/descriptions_train', 10000)
test_descs  = parse_descriptions('cs5785-fall19-final/descriptions_test', 2000)

train_descs = preprocessing(train_descs)
test_descs  = preprocessing(test_descs)

# TAGS

train_tags = parse_tags(parse_descriptions("cs5785-fall19-final/tags_train", num_doc=10000))
test_tags = parse_tags(parse_descriptions("cs5785-fall19-final/tags_test", num_doc=2000))

#### Features

In [31]:
train_feat = pd.read_csv("cs5785-fall19-final/features_train/features_resnet1000_train.csv", header = None, index_col=None)
test_feat = pd.read_csv("cs5785-fall19-final/features_test/features_resnet1000_test.csv", header = None, index_col=None)

train_feat_intern = pd.read_csv("cs5785-fall19-final/features_train/features_resnet1000intermediate_train.csv", header = None, index_col=None)
test_feat_intern = pd.read_csv("cs5785-fall19-final/features_test/features_resnet1000intermediate_test.csv", header = None, index_col=None)

In [32]:
for i in range(len(train_feat[0])):
    train_feat[0][i] = int(train_feat[0][i].replace("images_train/", "").replace(".jpg", ""))    
train_feat_sort = train_feat.sort_values(by=0)

for i in range(len(test_feat[0])):
    test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))
test_feat_sort = test_feat.sort_values(by=0)


for i in range(len(train_feat_intern[0])):
    train_feat_intern[0][i] = int(train_feat_intern[0][i].replace("images_train/", "").replace(".jpg", ""))    
train_feat_sort_intern = train_feat_intern.sort_values(by=0)

for i in range(len(test_feat_intern[0])):
    test_feat_intern[0][i] = int(test_feat_intern[0][i].replace("images_test/", "").replace(".jpg", ""))
test_feat_sort_intern = test_feat_intern.sort_values(by=0)



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-package

In [33]:
train_feat_sort = train_feat_sort.drop(columns=0).to_numpy()
test_feat_sort = test_feat_sort.drop(columns=0).to_numpy()


train_feat_sort_intern = train_feat_sort_intern.drop(columns=0).to_numpy()
test_feat_sort_intern = test_feat_sort_intern.drop(columns=0).to_numpy()

In [34]:
#pd.DataFrame(train_feat_sort_intern).to_csv("train_feat_sort_intern.csv")
#pd.DataFrame(test_feat_sort_intern).to_csv("test_feat_sort_intern.csv")


In [7]:
## DATA NORMALIZATION
train_feat_sort = StandardScaler(train_feat_sort)
test_feat_sort = StandardScaler(test_feat_sort)

## Word2vec

In [8]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)

#def doc_to_vec(sentence, word2vec):
#    word_vecs = [word2vec.get_vector(w) for w in sentence.split() if w in word2vec.vocab]
#    return np.stack(word_vecs).mean(0) 

In [57]:
# WORD2VEC CONCATENATING MAX AND MIN
def doc_to_vec(sentence, word2vec):
    word_vecs = [word2vec.get_vector(w) for w in sentence.split() if w in word2vec.vocab]
    #maxim = np.stack(word_vecs).max(0)
    #minim = np.stack(word_vecs).max(0)
    #return np.concatenate((maxim, minim), axis=0)
    return np.stack(word_vecs).max(0) + np.stack(word_vecs).max(0)

In [58]:
# DESCRIPTIONS
x_train = np.array([doc_to_vec(train_descs[i], word2vec) for i in range(len(train_descs))])
x_test = np.array([doc_to_vec(d, word2vec) for d in test_descs])

# DATA NORMALIZATION
x_train = StandardScaler(x_train)
x_test = StandardScaler(x_test)

In [59]:
# TAGS
x_train_tags = np.array([doc_to_vec(train_descs[i], word2vec) for i in range(len(train_tags))])
x_test_tags = np.array([doc_to_vec(d, word2vec) for d in test_tags])

# DATA NORMALIZATION
x_train_tags = StandardScaler(x_train_tags)
x_test_tags = StandardScaler(x_test_tags)

## PCA - Features

In [60]:
pca = PCA(n_components = 70)
pca.fit(train_feat_sort)

PCA(copy=True, iterated_power='auto', n_components=70, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [61]:
y_train = pca.transform(train_feat_sort)
y_test = pca.transform(test_feat_sort)

# Model(s) Training

## 1. MLP

In [205]:
xtrain, xval, ytrain, yval = train_test_split(x_train, y_train)

In [206]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='adam', alpha=1e-5, hidden_layer_sizes=(100,), random_state=1, max_iter = 1000)
clf.fit(xtrain, ytrain)
yval_pred = clf.predict(xval)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [207]:
'''
RESULTS: 
Development MAP@20: 0.15715323095892755 [adam, hidden_layer_sizes = (100,) // PCA = 100 components]


'''

'\nRESULTS: \nDevelopment MAP@20: 0.15715323095892755 [adam, hidden_layer_sizes = (100,) // PCA = 100 components]\n\n'

## 2. Ridge

In [62]:
xtrain, xval, ytrain, yval = train_test_split(x_train, y_train)

In [63]:
from sklearn.kernel_ridge import KernelRidge

def kernel_ridge(xtrain, ytrain):
    kr = KernelRidge()
#     kr.fit(train_bow, train_feat_pca)
    kr.fit(xtrain, ytrain)
    return kr

In [64]:
# CODE NEEDED TO RUN ALL THE CODE
# kr = kernel_ridge(train_bow, train_feat_pca)
# preds = kr.predict(test_bow)
# dm = dist_matrix(preds, test_feat_pca)
# top_images = output_format(dm)
# outputCSV(top_images)

In [65]:
kr2 = kernel_ridge(xtrain, ytrain)
yval_pred = kr2.predict(xval)

In [53]:
'''
Development MAP@20: 0.18237442016359973 [PCA = 50 components]
Development MAP@20: 0.1928452635084214  [PCA = 70 components] ** BEST
Development MAP@20: 0.19259939057502526 [PCA = 100 components]
Development MAP@20: 0.19369393946438837 [PCA = 100 components]
Development MAP@20: 0.18889584037737286 [PCA = 300 components]

Concatenating Max and Min in word2vec:
Development MAP@20: 0.142785998571432 [PCA = 70 components]

Adding Max and Min in word2vec:
Development MAP@20: 0.11136266005752075 [PCA = 70 components]


'''

'\nDevelopment MAP@20: 0.18237442016359973 [PCA = 50 components]\nDevelopment MAP@20: 0.1928452635084214  [PCA = 70 components] ** BEST\nDevelopment MAP@20: 0.19259939057502526 [PCA = 100 components]\nDevelopment MAP@20: 0.19369393946438837 [PCA = 100 components]\nDevelopment MAP@20: 0.18889584037737286 [PCA = 300 components]\n\n\n'

## 3. Ridge with Tags - WRONG APPROACH! Tags belong to images. Needs to be changed. However, why did I get this accuracy?!?!? 

In [312]:
xtrain, xval, ytrain, yval = train_test_split(x_train_tags, y_train)

In [313]:
from sklearn.kernel_ridge import KernelRidge

def kernel_ridge(xtrain, ytrain):
    kr = KernelRidge()
#     kr.fit(train_bow, train_feat_pca)
    kr.fit(xtrain, ytrain)
    return kr

In [314]:
kr2 = kernel_ridge(xtrain, ytrain)
yval_pred = kr2.predict(xval)

In [295]:
'''
Development MAP@20: 0.18799833563753068 [PCA = 70 components]
'''

'\nDevelopment MAP@20: 0.18203206580485218 [PCA = ]\n'

## Accuracy in the validation set

In [66]:
def dist_matrix(x1, x2):
    return cdist(x1,x2,'cosine')

In [67]:
val_distances = dist_matrix(yval_pred, yval)

In [68]:
val_scores = []
val_pos_list = []

for i in range(len(yval)):
    pred_dist_idx = list(np.argsort(val_distances[i]))
    val_pos = pred_dist_idx.index(i)
    val_pos_list.append(val_pos)
    if val_pos < 20:
        val_scores.append(1 / (val_pos + 1))
    else:
        val_scores.append(0.0)

print("Development MAP@20:", np.mean(val_scores))
print("Mean index of true image", np.mean(val_pos_list))
print("Median index of true image", np.median(val_pos_list))

Development MAP@20: 0.11136266005752075
Mean index of true image 104.3556
Median index of true image 33.0
